## Sprintの目的
>- 自然言語処理の一連の流れを学ぶ
>- 自然言語のベクトル化の方法を学ぶ

## 自然言語のベクトル化
>**自然言語処理**（**NLP**, **Natural Language Processing**） とは人間が普段使っている 自然言語 をコンピュータに処理させる技術のことです。ここではその中でも、機械学習の入力として自然言語を用いることを考えていきます。
>
>多くの機械学習手法は **数値データ**（**量的変数**） の入力を前提にしていますので、`自然言語の **テキストデータ** を数値データに変換する必要があります。`これを **自然言語のベクトル化** と呼びます。ベクトル化の際にテキストデータの特徴をうまく捉えられるよう、様々な手法が考えられてきていますので、このSprintではそれらを学びます。



## 非構造化データ
>データの分類として、表に数値がまとめられたような`コンピュータが扱いやすい形`を **構造化データ** 、人間が扱いやすい画像・動画・テキスト・音声などを **非構造化データ** と呼ぶことがあります。  
>自然言語のベクトル化は、`非構造化データを構造化データに変換する工程`と言えます。同じ非構造化データでも、画像に対してはディープラーニングを用いる場合この変換作業はあまり必要がありませんでしたが、`テキストにおいてはこれをどう行うかが重要`です。

## 自然言語処理により何ができるか
>機械学習の入力や出力に自然言語のテキストを用いることで様々なことができます。入力も出力もテキストである例としては **機械翻訳** があげられ、実用化されています。入力は画像で出力がテキストである **画像キャプション生成** やその逆の文章からの画像生成も研究が進んでいます。
>
>しかし、出力をテキストや画像のような非構造化データとすることは`難易度が高い`です。`比較的簡単にできることとしては、入力をテキスト、出力をカテゴリーとする **テキスト分類** です`。
>
>アヤメやタイタニック、手書き数字のような定番の存在として、**IMDB映画レビューデータセット** の感情分析があります。レビューの文書が映画に対して`肯定的か否定的かを2値分類`します。文書ごとの肯定・否定はラベルが与えられています。このSprintではこれを使っていきます。

## IMDB映画レビューデータセットの準備

## ダウンロード
>次のwgetコマンドによってダウンロードします。
>
>以下のサイトで公開されているデータセットです。
>Sentiment Analysis

In [1]:
#!brew install wget

In [2]:
# IMDBをカレントフォルダにダウンロード
!wget http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
# 解凍
!tar zxf aclImdb_v1.tar.gz
# aclImdb/train/unsupはラベル無しのため削除
!rm -rf aclImdb/train/unsup
# IMDBデータセットの説明を表示
!cat aclImdb/README

--2020-12-25 00:57:12--  http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
ai.stanford.edu (ai.stanford.edu) をDNSに問いあわせています... 171.64.68.10
ai.stanford.edu (ai.stanford.edu)|171.64.68.10|:80 に接続しています... 接続しました。
HTTP による接続要求を送信しました、応答を待っています... 200 OK
長さ: 84125825 (80M) [application/x-gzip]
`aclImdb_v1.tar.gz.11' に保存中

aclImdb_v1.tar.gz.1 100%[===================>]  80.23M  16.4MB/s 時間 6.8s       

2020-12-25 00:57:20 (11.8 MB/s) - `aclImdb_v1.tar.gz.11' へ保存完了 [84125825/84125825]

Large Movie Review Dataset v1.0

Overview

This dataset contains movie reviews along with their associated binary
sentiment polarity labels. It is intended to serve as a benchmark for
sentiment classification. This document outlines how the dataset was
gathered, and how to use the files provided. 

Dataset 

The core dataset contains 50,000 reviews split evenly into 25k train
and 25k test sets. The overall distribution of labels is balanced (25k
pos and 25k neg). We also include an additional 50,0

## 読み込み  
>scikit-learnのload_filesを用いて読み込みます。
>
>sklearn.datasets.load_files — scikit-learn 0.21.3 documentation
>
>《読み込むコード》

In [3]:
from sklearn.datasets import load_files

train_review = load_files('./aclImdb/train/', encoding='utf-8')
x_train, y_train = train_review.data, train_review.target

test_review = load_files('./aclImdb/test/', encoding='utf-8')
x_test, y_test = test_review.data, test_review.target

# ラベルの0,1と意味の対応の表示
print(train_review.target_names)

['neg', 'pos']


In [4]:
import numpy as np

In [5]:
print(np.array(x_train).shape)
print(np.array(y_train).shape)
print(np.array(x_test).shape)
print(np.array(y_test).shape)

(25000,)
(25000,)
(25000,)
(25000,)


## このデータセットについて
>中身を見てみると、英語の文章が入っていることが分かります。

In [6]:
print("x : {}".format(x_train[0]))

x : Zero Day leads you to think, even re-think why two boys/young men would do what they did - commit mutual suicide via slaughtering their classmates. It captures what must be beyond a bizarre mode of being for two humans who have decided to withdraw from common civility in order to define their own/mutual world via coupled destruction.<br /><br />It is not a perfect movie but given what money/time the filmmaker and actors had - it is a remarkable product. In terms of explaining the motives and actions of the two young suicide/murderers it is better than 'Elephant' - in terms of being a film that gets under our 'rationalistic' skin it is a far, far better film than almost anything you are likely to see. <br /><br />Flawed but honest with a terrible honesty.


>IMDBはInternet Movie Databaseの略で、映画のデータベースサイトです。
>
>Ratings and Reviews for New Movies and TV Shows - IMDb
>
>このサイトではユーザーが映画に対して`1から10点の評価とコメント`を投稿することができます。そのデータベースから訓練データは25000件、テストデータは25000件のデータセットを作成しています。
>
>`4点以下を否定的、7点以下を肯定的なレビューとして2値のラベル付け`しており、これにより感情の分類を行います。5,6点の中立的なレビューはデータセットに含んでいません。また、ラベルは訓練用・テスト用それぞれで均一に入っています。詳細はダウンロードしたREADMEを確認してください。

## 古典的な手法
>古典的ながら現在でも強力な手法であるBoWとTF-IDFを見ていきます。

## BoW

>単純ながら効果的な方法として **BoW (Bag of Words)** があります。
>これは、サンプルごとに単語などの **登場回数** を数えたものをベクトルとする方法です。
>単語をカテゴリとして捉え **one-hot表現** していることになります。



## 例
>例として、IMDBデータセットからある3文の最初の5単語を抜き出したものを用意しました。

In [7]:
mini_dataset = \
  ["This movie is very good.",
  "This film is a good",
  "Very bad. Very, very bad."]

>この3文にBoWを適用させてみます。scikit-learnのCountVectorizerを利用します。
>
>sklearn.feature_extraction.text.CountVectorizer — scikit-learn 0.21.3 documentation




In [8]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(token_pattern=r'(?u)\b\w+\b')
bow = (vectorizer.fit_transform(mini_dataset)).toarray()

# DataFrameにまとめる
df = pd.DataFrame(bow, columns=vectorizer.get_feature_names())
display(df)

,a,bad,film,good,is,movie,this,very
0,0,0,0,1,1,1,1,1
1,1,0,1,1,1,0,1,0
2,0,2,0,0,0,0,0,3


>例にあげた`3文の中で登場する8種類の単語が列名`になり、`0,1,2番目のサンプルでそれらが何回登場しているか`を示しています。2番目のサンプル「Very bad. Very, very bad.」ではbadが2回、veryが3回登場しています。列名になっている言葉はデータセットが持つ **語彙** と呼びます。
>
>テキストはBoWにより各サンプルが語彙数の次元を持つ特徴量となり、機械学習モデルへ入力できるようになります。`この時使用したテキスト全体`のことを **コーパス** と呼びます。`語彙はコーパスに含まれる言葉よって決まり、それを特徴量としてモデルの学習`を行います。そのため、テストデータではじめて登場する語彙はベクトル化される際に無視されます。



## 前処理  
>CountVectorizerクラスでは大文字は小文字に揃えるという **前処理** が自動的に行われています。こういった前処理は自然言語処理において大切で、不要な記号などの消去（**テキストクリーニング**）や表記揺れの統一といったことを別途行うことが一般的です。
>
>語形が「see」「saw」「seen」のように変化する単語に対して語幹に揃える **ステミング** と呼ばれる処理を行うこともあります。



## トークン
>`BoWは厳密には単語を数えているのではなく、 **トークン（token）** として定めた固まりを数えます`。
>
>何をトークンとするかは**CountVectorizer**では引数`token_pattern`で **正規表現** の記法により指定されます。デフォルトは`r'(?u)\b\w\w+\b'`ですが、上の例では`r'(?u)\b\w+\b'`としています。
>
>デフォルトでは空白・句読点・スラッシュなどに囲まれた2文字以上の文字を1つのトークンとして抜き出すようになっているため、「a」や「I」などがカウントされません。  
>英語では1文字の単語は文章の特徴をあまり表さないため、除外されることもあります。しかし、上の例では1文字の単語もトークンとして抜き出すように引数を指定しています。

#### 《正規表現》

>正規表現は前処理の際にも活用しますが、ここでは詳細は扱いません。Pythonではreモジュールによって正規表現操作ができます。
>
>re — 正規表現操作
>
>正規表現を利用する際はリアルタイムで結果を確認できる以下のようなサービスが便利です。
>
>Online regex tester and debugger: PHP, PCRE, Python, Golang and JavaScript



## 形態素解析
>英語などの多くの言語では`空白`という分かりやすい基準でトークン化が行えますが、日本語ではそれが行えません。
>
>`日本語では名詞や助詞、動詞のように異なる **品詞** で分けられる単位で **分かち書き** することになります`。例えば「私はプログラミングを学びます」という日本語の文は「私/は/プログラミング/を/学び/ます」という風になります。
>
>`これには **MeCab** や **Janome** のような形態素解析ツールを用います`。Pythonから利用することも可能です。MeCabをウェブ上で簡単に利用できる**Web茶まめ**というサービスも国立国語研究所が提供しています。
>
>自然言語では`新しい言葉`も日々生まれますので、それにどれだけ対応できるかも大切です。MeCab用の毎週更新される辞書として **mecab-ipadic-NEologd** がオープンソースで存在しています。
>
>mecab-ipadic-neologd/README.ja.md at master · neologd/mecab-ipadic-neologd



## n-gram
>上のBoWの例では1つの単語（トークン）毎の登場回数を数えましたが、これでは`語順`は全く考慮されていません。
>
>考慮するために、`隣あう単語同士をまとめて扱う **n-gram** `という考え方を適用することがあります。2つの単語をまとめる場合は **2-gram (bigram)** と呼び、次のようになります。



In [9]:
# ngram_rangeで利用するn-gramの範囲を指定する
vectorizer = CountVectorizer(ngram_range=(2, 2), token_pattern=r'(?u)\b\w+\b')
bow_train = (vectorizer.fit_transform(mini_dataset)).toarray()
df = pd.DataFrame(bow_train, columns=vectorizer.get_feature_names())
display(df)

,a good,bad very,film is,is a,is very,movie is,this film,this movie,very bad,very good,very very
0,0,0,0,0,1,1,0,1,0,1,0
1,1,0,1,1,0,0,1,0,0,0,0
2,0,1,0,0,0,0,0,0,2,0,1


>2-gramにより「very good」と「very bad」が区別して数えられています。
>
>単語をまとめない場合は **1-gram (unigram) **と呼びます。3つまとめる3-gram(trigram)など任意の数を考えることができます。1-gramと2-gramを組み合わせてBoWを行うといったこともあります。



## 【問題1】BoWのスクラッチ実装  
>以下の3文のBoWを求められるプログラムをscikit-learnを使わずに作成してください。1-gramと2-gramで計算してください。

>This movie is SOOOO funny!!!  
>What a movie! I never  
>best movie ever!!!!! this movie  

In [10]:
import string
import pandas as pd


def scratch_Bow(sentence, n):
    """
    Bowのスクラッチ
    
    Parameters
    ----------------
    sentence: list（string）
      Bowの対象となる文章
    n: int
      n-gramのn

    Returns
    ----------------
    
    """
    # 正規表現の準備
    kigo = string.punctuation# >>!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~
    table = str.maketrans( "", "", kigo)

    # 全部小文字にする
    sentence = list(map(str.lower,sentence))
    
    #文ごとに単語単位に分ける
    sentence_words = []
    for s in sentence:
        result = s.translate(table)#正規表現
        sentence_words.append(result.split(' '))
        
    ############ n-gram ############
    sentence_words_n_gram = []
    for s in sentence_words:
        n_gram_temp = []
        for i in range(len(s)-n+1):
            n_gram_temp.append(' '.join(s[i:i+n]))
        sentence_words_n_gram.append(n_gram_temp)

    # 特徴量用に平坦化する
    feature_list = sum(sentence_words_n_gram, [])
    
    # BoWの表の作成
    n_appearances = []
    for s in sentence_words_n_gram:
        n_app_sentence = []
        for f in feature_list:
            n_app_sentence.append(s.count(f))
        n_appearances.append(n_app_sentence)

    df = pd.DataFrame(n_appearances, columns=feature_list)
    display(df)


# 分類したい文字列
sentence = ['This movie is SOOOO funny!!!',
                        'What a movie! I never',
                        'best movie ever!!!!! this movie'
                       ]

n=1
scratch_Bow(sentence, n)

n=2
scratch_Bow(sentence, n)


,this,movie,is,soooo,funny,what,a,movie,i,never,best,movie,ever,this,movie
0,1,1,1,1,1,0,0,1,0,0,0,1,0,1,1
1,0,1,0,0,0,1,1,1,1,1,0,1,0,0,1
2,1,2,0,0,0,0,0,2,0,0,1,2,1,1,2


,this movie,movie is,is soooo,soooo funny,what a,a movie,movie i,i never,best movie,movie ever,ever this,this movie
0,1,1,1,1,0,0,0,0,0,0,0,1
1,0,0,0,0,1,1,1,1,0,0,0,0
2,1,0,0,0,0,0,0,0,1,1,1,1


## TF-IDF

>BoWの発展的手法として **TF-IDF** もよく使われます。これは **Term Frequency (TF)** と **Inverse Document Frequency (IDF)** という2つの指標の組み合わせです。
>
>《標準的なTF-IDFの式》
>
>
>Term Frequency:
>$
tf(t,d) = \frac{n_{t,d}}{\sum_{s \in d}n_{s,d}}
$
>
>
>$n_{t,d}$ : サンプルd内のトークンtの出現回数（BoWと同じ）
>
>$\sum_{s \in d}n_{s,d}$ : サンプルdの全トークンの出現回数の和
>
>
>Inverse Document Frequency:
>$
idf(t) = \log{\frac{N}{df(t)}}
$
>
>$N$ : サンプル数
>
>$df(t)$ : トークンtが出現するサンプル数
>
>＊logの底は任意の値
>
>
>TF-IDF:
>
>$
tfidf(t, d) = tf(t, d) \times idf(t)
$

## IDF
>IDFは`そのトークンがデータセット内で珍しい`ほど値が大きくなる指標です。
>
>サンプル数 $N$ をIMDB映画レビューデータセットの訓練データに合わせ25000として、トークンが出現するサンプル数 $df(t)$ を変化させたグラフを確認してみると、次のようになります。



In [11]:
import numpy as np
import matplotlib.pyplot as plt

n_samples = 25000
idf = np.log(n_samples/np.arange(1,n_samples))
plt.title("IDF")
plt.xlabel("df(t)")
plt.ylabel("IDF")
plt.plot(idf)
plt.show()

<Figure size 640x480 with 1 Axes>

>TF-IDFではこの数を`出現回数に掛け合わせる`ので、`珍しいトークンの登場に重み付け`を行なっていることになります。

## ストップワード
>`あまりにも頻繁に登場するトークン`は、値を小さくするだけでなく、`取り除くという前処理`を加えることもあります。  
>取り除くもののことを **ストップワード** と呼びます。  
>既存の**ストップワード一覧**を利用したり、しきい値によって求めたりします。  
>
>scikit-learnのCountVectorizerでは引数`stop_words`に**リストで指定**することで処理を行なってくれます。

In [12]:
vectorizer = CountVectorizer(stop_words=["is"], token_pattern=r'\b\w+\b')
bow_train = (vectorizer.fit_transform(mini_dataset)).toarray()
df = pd.DataFrame(bow_train, columns=vectorizer.get_feature_names())
display(df)

,a,bad,film,good,movie,this,very
0,0,0,0,1,1,1,1
1,1,0,1,1,0,1,0
2,0,2,0,0,0,0,3


>代表的な既存のストップワード一覧としては、NLTK という自然言語処理のライブラリのものがあげられます。
>あるデータセットにおいては特別重要な意味を持つ単語が一覧に含まれている可能性もあるため、使用する際は中身を確認することが望ましいです。



In [13]:
# はじめて使う場合はストップワードをダウンロード
import nltk
stop_words = nltk.download('stopwords')

from nltk.corpus import stopwords
stop_words = stopwords.words('english')

print("stop word : {}".format(stop_words)) # 'i', 'me', 'my', ...

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/ishiitomoaki/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
stop word : ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when'

>逆に、`登場回数が特に少ないトークンも取り除くことが多い`です。全てのトークンを用いるとベクトルの次元数が著しく大きくなってしまい計算コストが高まるためです。
>
>scikit-learnのCountVectorizerでは引数`max_featuresに最大の語彙数を指定`することで処理を行なってくれます。以下の例では出現数が多い順に5個でベクトル化しています。



In [14]:
vectorizer = CountVectorizer(token_pattern=r'\b\w+\b', max_features = 5)
bow_train = (vectorizer.fit_transform(mini_dataset)).toarray()
df = pd.DataFrame(bow_train, columns=vectorizer.get_feature_names())
display(df)

,bad,good,is,this,very
0,0,1,1,1,1
1,0,1,1,1,0
2,2,0,0,0,3


## 【問題2】TF-IDFの計算
>IMDB映画レビューデータセットをTF-IDFによりベクトル化してください。  
>NLTKのストップワードを利用し、最大の語彙数は5000程度に設定してください。  
>テキストクリーニングやステミングなどの前処理はこの問題では要求しません。  
>
>TF-IDFの計算にはscikit-learnの以下のどちらかのクラスを使用してください。  
>
>sklearn.feature_extraction.text.TfidfVectorizer — scikit-learn 0.21.3 documentation  
>sklearn.feature_extraction.text.TfidfTransformer — scikit-learn 0.21.3 documentation  
>
>なお、scikit-learnでは標準的な式とは異なる式が採用されています。  
>
>また、デフォルトではnorm="l2"の引数が設定されており、各サンプルにL2正規化が行われます。norm=Noneとすることで正規化は行われなくなります。  
>
>
>Term Frequency:  
>
>$
tf(t,d) = n_{t,d}
$
>
>$n_{t,d}$ : サンプルd内のトークンtの出現回数
>
>scikit-learnのTFは分母がなくなりBoWと同じ計算になります。
>
>
>Inverse Document Frequency:
>
>$
idf(t) = \log{\frac{1+N}{1+df(t)}}+1
$
>
>$N$ : サンプル数
>
>$df(t)$ : トークンtが出現するサンプル数
>
>＊logの底はネイピア数e
>
>詳細は以下のドキュメントを確認してください。
>
>5.2.3.4. Tf–idf term weighting — scikit-learn 0.21.3 documentation

In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(stop_words=["is"], token_pattern=r'\b\w+\b', max_features = 5000)
X = vectorizer.fit_transform(x_train)

print(vectorizer.get_feature_names())
print(X.shape)

['0', '00', '000', '1', '10', '100', '11', '12', '13', '13th', '14', '15', '16', '17', '18', '1930', '1930s', '1933', '1940', '1950', '1950s', '1960', '1960s', '1968', '1970', '1970s', '1972', '1973', '1980', '1980s', '1983', '1984', '1990', '1996', '1999', '1st', '2', '20', '2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007', '20th', '24', '25', '2nd', '3', '30', '35', '3d', '3rd', '4', '40', '45', '5', '50', '50s', '6', '60', '60s', '7', '70', '70s', '8', '80', '80s', '9', '90', '90s', '99', 'a', 'abandoned', 'abc', 'abilities', 'ability', 'able', 'about', 'above', 'abraham', 'absence', 'absolute', 'absolutely', 'absurd', 'abuse', 'abusive', 'abysmal', 'academy', 'accent', 'accents', 'accept', 'acceptable', 'accepted', 'access', 'accident', 'accidentally', 'accompanied', 'accomplished', 'according', 'account', 'accurate', 'accused', 'achieve', 'achieved', 'achievement', 'acid', 'across', 'act', 'acted', 'acting', 'action', 'actions', 'actor', 'actors', 'actress', 'actresse

## 【問題3】TF-IDFを用いた学習
>問題2で求めたベクトルを用いてIMDB映画レビューデータセットの学習・推定を行なってください。モデルは2値分類が行える任意のものを利用してください。
>
>ここでは精度の高さは求めませんが、最大の語彙数やストップワード、n-gramの数を変化させて影響を検証してみてください。

In [16]:
from sklearn.model_selection import train_test_split

y = np.copy(y_train)

# 学習データとテストデータを７５：２５で分割
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.25, random_state=0)

print(X_train.shape)
print(y_train.shape)
print(X_val.shape)
print(y_val.shape)


(18750, 5000)
(18750,)
(6250, 5000)
(6250,)


In [17]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [18]:
# stop_words=["is"], token_pattern=r'\b\w+\b', max_features = 5000

logistic = LogisticRegression(random_state=0)
logistic.fit(X_train, y_train)

y_pred = logistic.predict(X_val)
print("ロジスティック回帰による予測値\n{}".format(y_pred))

acc = accuracy_score(y_val, y_pred)
print("ロジスティック回帰の正解率： {}".format(acc))

ロジスティック回帰による予測値
[0 0 1 ... 0 0 0]
ロジスティック回帰の正解率： 0.88448


In [19]:
# max_features = 10000 に変更

vectorizer = TfidfVectorizer(stop_words=["is"], token_pattern=r'\b\w+\b', max_features = 10000)
X = vectorizer.fit_transform(x_train)

# 学習データとテストデータを７５：２５で分割
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.25, random_state=0)

logistic = LogisticRegression(random_state=0)
logistic.fit(X_train, y_train)

y_pred = logistic.predict(X_val)
print("max_features = 10000　の予測値\n{}".format(y_pred))

acc = accuracy_score(y_val, y_pred)
print("max_features = 10000　の正解率： {}".format(acc))

max_features = 10000　の予測値
[0 0 1 ... 0 0 0]
max_features = 10000　の正解率： 0.88848


In [20]:
# stop_words=["is", "was", "the", "this"] に変更

vectorizer = TfidfVectorizer(stop_words=["is", "was", "the", "this"], token_pattern=r'\b\w+\b', max_features = 5000)
X = vectorizer.fit_transform(x_train)

# 学習データとテストデータを７５：２５で分割
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.25, random_state=0)

logistic = LogisticRegression(random_state=0)
logistic.fit(X_train, y_train)

y_pred = logistic.predict(X_val)
print("stop_words の予測値\n{}".format(y_pred))

acc = accuracy_score(y_val, y_pred)
print("stop_words の正解率： {}".format(acc))

stop_words の予測値
[0 0 1 ... 0 0 0]
stop_words の正解率： 0.88416


In [21]:
# ngram_range=(2, 4) を追加

vectorizer = TfidfVectorizer(ngram_range=(2, 4), stop_words=["is"], token_pattern=r'\b\w+\b', max_features = 5000)
X = vectorizer.fit_transform(x_train)

# 学習データとテストデータを７５：２５で分割
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.25, random_state=0)

logistic = LogisticRegression(random_state=0)
logistic.fit(X_train, y_train)

y_pred = logistic.predict(X_val)
print("ngram_range=(2, 4) の予測値\n{}".format(y_pred))

acc = accuracy_score(y_val, y_pred)
print("ngram_range=(2, 4) の正解率： {}".format(acc))

ngram_range=(2, 4) の予測値
[0 0 1 ... 1 0 0]
ngram_range=(2, 4) の正解率： 0.84176


## 【問題4】TF-IDFのスクラッチ実装
>以下の3文のTF-IDFを求められるプログラムをscikit-learnを使わずに作成してください。標準的な式と、scikit-learnの採用している式の2種類を作成してください。正規化は不要です。

In [22]:
import string
import pandas as pd


def scratch_TF_IDF(sentence, n):
    """
    Bowのスクラッチ
    
    Parameters
    ----------------
    sentence: list（string）
      Bowの対象となる文章
    n: int
      n-gramのn

    Returns
    ----------------
    
    """
    # 正規表現の準備
    kigo = string.punctuation# >>!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~
    table = str.maketrans( "", "", kigo)

    # 全部小文字にする
    sentence = list(map(str.lower,sentence))
    
    #文ごとに単語単位に分ける
    sentence_words = []
    for s in sentence:
        result = s.translate(table)#正規表現
        sentence_words.append(result.split(' '))
        
    ############ n-gram ############
    sentence_words_n_gram = []
    for s in sentence_words:
        n_gram_temp = []
        for i in range(len(s)-n+1):
            n_gram_temp.append(' '.join(s[i:i+n]))
        sentence_words_n_gram.append(n_gram_temp)

    # 特徴量用に平坦化する
    feature_list = sum(sentence_words_n_gram, [])
    
    # BoWの表の作成
    n_appearances = []
    tf = []
    for s in sentence_words_n_gram:
        n_app_sentence = []
        n_td = []
        for f in feature_list:
            n_app_sentence.append(s.count(f))
        n_appearances.append(n_app_sentence)
        ########## TF の　計算　##########
        n_td = np.array(n_app_sentence)
        tf.append(n_td/np.sum(n_td))

    tf = np.array(tf)
    n_appearances= np.array(n_appearances)
    
    ########## IDF の　計算　##########
    idf = np.log((1+len(sentence_words))/(1+np.sum(tf, axis=0)))+1
    
    ########## TF-IDF の　実装　##########
    tf_idf = tf*idf
        
    df = pd.DataFrame(tf_idf, columns=feature_list)
    display(df)

    
    
    
# 分類したい文字列
sentence = ['This movie is SOOOO funny!!!',
                        'What a movie! I never',
                        'best movie ever!!!!! this movie'
                       ]

n=1
scratch_TF_IDF(sentence, n)

n=2
scratch_TF_IDF(sentence, n)


,this,movie,is,soooo,funny,what,a,movie,i,never,best,movie,ever,this,movie
0,0.245401,0.227538,0.253437,0.253437,0.253437,0.000000,0.000000,0.227538,0.000000,0.000000,0.000000,0.227538,0.000000,0.245401,0.227538
1,0.000000,0.255980,0.000000,0.000000,0.000000,0.283564,0.283564,0.255980,0.283564,0.283564,0.000000,0.255980,0.000000,0.000000,0.255980
2,0.184051,0.341307,0.000000,0.000000,0.000000,0.000000,0.000000,0.341307,0.000000,0.000000,0.192188,0.341307,0.192188,0.184051,0.341307


,this movie,movie is,is soooo,soooo funny,what a,a movie,movie i,i never,best movie,movie ever,ever this,this movie
0,0.409964,0.440795,0.440795,0.440795,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.409964
1,0.000000,0.000000,0.000000,0.000000,0.540788,0.540788,0.540788,0.540788,0.000000,0.000000,0.000000,0.000000
2,0.409964,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.440795,0.440795,0.440795,0.409964


In [23]:
# scikt-learn

import string
import pandas as pd


def scratch_TF_IDF(sentence, n):
    """
    Bowのスクラッチ
    
    Parameters
    ----------------
    sentence: list（string）
      Bowの対象となる文章
    n: int
      n-gramのn

    Returns
    ----------------
    
    """
    # 正規表現の準備
    kigo = string.punctuation# >>!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~
    table = str.maketrans( "", "", kigo)

    # 全部小文字にする
    sentence = list(map(str.lower,sentence))
    
    #文ごとに単語単位に分ける
    sentence_words = []
    for s in sentence:
        result = s.translate(table)#正規表現
        sentence_words.append(result.split(' '))
        
    ############ n-gram ############
    sentence_words_n_gram = []
    for s in sentence_words:
        n_gram_temp = []
        for i in range(len(s)-n+1):
            n_gram_temp.append(' '.join(s[i:i+n]))
        sentence_words_n_gram.append(n_gram_temp)

    # 特徴量用に平坦化する
    feature_list = sum(sentence_words_n_gram, [])
    
    # BoWの表の作成
    n_appearances = []
    tf = []
    for s in sentence_words_n_gram:
        n_app_sentence = []
        n_td = []
        for f in feature_list:
            n_app_sentence.append(s.count(f))
        n_appearances.append(n_app_sentence)
        ########## TF の　計算　##########
        n_td = np.array(n_app_sentence)
        tf.append(n_td/np.sum(n_td))

    tf = np.array(tf)
    n_appearances= np.array(n_appearances)
    
    ########## IDF の　計算　##########
    # 分母を削除
    idf = np.log((1+len(sentence_words)))+1
    
    ########## TF-IDF の　実装　##########
    tf_idf = tf*idf
        
    df = pd.DataFrame(tf_idf, columns=feature_list)
    display(df)

    
    
    
# 分類したい文字列
sentence = ['This movie is SOOOO funny!!!',
                        'What a movie! I never',
                        'best movie ever!!!!! this movie'
                       ]

n=1
scratch_TF_IDF(sentence, n)

n=2
scratch_TF_IDF(sentence, n)


,this,movie,is,soooo,funny,what,a,movie,i,never,best,movie,ever,this,movie
0,0.265144,0.265144,0.265144,0.265144,0.265144,0.000000,0.000000,0.265144,0.000000,0.000000,0.000000,0.265144,0.000000,0.265144,0.265144
1,0.000000,0.298287,0.000000,0.000000,0.000000,0.298287,0.298287,0.298287,0.298287,0.298287,0.000000,0.298287,0.000000,0.000000,0.298287
2,0.198858,0.397716,0.000000,0.000000,0.000000,0.000000,0.000000,0.397716,0.000000,0.000000,0.198858,0.397716,0.198858,0.198858,0.397716


,this movie,movie is,is soooo,soooo funny,what a,a movie,movie i,i never,best movie,movie ever,ever this,this movie
0,0.477259,0.477259,0.477259,0.477259,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.477259
1,0.000000,0.000000,0.000000,0.000000,0.596574,0.596574,0.596574,0.596574,0.000000,0.000000,0.000000,0.000000
2,0.477259,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.477259,0.477259,0.477259,0.477259
